In [15]:
import pandas as pd
import numpy as np

In [16]:
Data1=pd.read_excel('../Database/제주관광공사_제주관광정보시스템(VISIT JEJU)_콘텐츠_20230315.xlsx') #제주도 전체 콘텐츠 데이터
Data3=pd.read_csv('../Database/NXE_3rd_ORIGIN_NXEGGRESTRTAL_20210101000000.csv') #음식점 평점
Data4=pd.read_csv('../Database/NXE_3rd_ORIGIN_NXEGGSTAYNGAL_20210101000000.csv') #숙박업 평점
Data5=pd.read_csv('../Database/NXE_3rd_ORIGIN_NXEGGTRRSRTAL_20210101000000.csv') #관광지 평점

data1=pd.DataFrame(Data1)
data3=pd.DataFrame(Data3)
data4=pd.DataFrame(Data4)
data5=pd.DataFrame(Data5)

add=[]
for x in [data3,data4,data5]:
    add.append(list(x.columns)) 
    
add=list(map(lambda x: dict(zip(["이름","구분","평점","day1"],x)),add))

result=[]
for x in [data3,data4,data5]:
    result.append(x.rename(columns=dict(zip(x.columns,["이름","구분","평점","구분1","day1","day2"]))).drop(columns=["구분1","day2"]))

for i in range(3):
    result[i]["평점"]=result[i]["평점"].astype("float") #평점 데이터가 str형태 -> 실수(float)으로 변경

아래 코딩을 음식점,관광지에 동일 적용

In [17]:
test1=pd.DataFrame(result[1].groupby("이름")["평점"].mean())
test1.index.name="기준"
test2=data1[data1["콘텐츠분류"]=="숙박"].set_index("제목") # 콘텐츠 분류는 따로 적용
test2.index.name="기준"
test3=pd.merge(test2,test1,how="left")
test3.to_excel('../수정데이터/숙박(평점포함).xlsx') # -> 임시 저장

In [18]:
result={}
for name in ["숙박","음식점","관광지"]:
    data=pd.read_excel(f"../수정데이터/{name}(평점포함).xlsx")
    drop_cols=[data.columns[i] for i in sum([list(range(26,42)),
                                         list(range(51,73))],[])]
    drop_cols.extend(sum([
    ["웹사용여부","모바일사용여부","작성상태",'사용여부','공개여부','트립어드바이저코드','템플릿여부'],
    ["콘텐츠아이디","연관콘텐츠","외부검색어",'번역요청일시'],
    ["언어","작성상태_1","수정자","수정일시","우편번호"],
    ["웹사용여부","모바일사용여부","작성상태",'사용여부','공개여부','트립어드바이저코드','템플릿여부']],[]))
    Use_data=data.drop(columns=drop_cols)
    Use_data=Use_data[(Use_data["지번주소"].isna()==False)&(Use_data["도로명주소"].isna()==False)]
    result[name]=Use_data[Use_data["콘텐츠분류"]==name]
for name in result:
    result[name].to_excel(f"../수정데이터/{name}(최종).xlsx")

In [21]:
for name in result:
    print(f"{name}")
    print("-"*30)
    print(result[name].info())
    print("-"*30)

숙박
------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 8422 entries, 0 to 8500
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  8422 non-null   int64         
 1   콘텐츠분류       8422 non-null   object        
 2   타이틀         8302 non-null   object        
 3   지번주소        8422 non-null   object        
 4   도로명주소       8422 non-null   object        
 5   위도          8382 non-null   float64       
 6   경도          8382 non-null   float64       
 7   탐나오연동여부     8421 non-null   object        
 8   평점          7715 non-null   float64       
 9   소셜네트워크주소    1453 non-null   object        
 10  조회수         8422 non-null   int64         
 11  좋아요수        8422 non-null   int64         
 12  리뷰수         8422 non-null   int64         
 13  북마크수        8422 non-null   int64         
 14  일정등록수       8422 non-null   int64         
 15  다녀왔어요수      8422 non-null   int64         


In [27]:
#제목이 없는 데이터 삭제(nan값 처리)
for name in result:
    print(f"{name}")
    print("Nan개수 : ",result[name]["타이틀"].isna().sum())
    print("-"*30)
    final=result[name][result[name]["타이틀"].isna()==False]
    final.to_excel(f"../수정데이터/{name}(최종).xlsx")

숙박
Nan개수 :  120
------------------------------
음식점
Nan개수 :  120
------------------------------
관광지
Nan개수 :  157
------------------------------
